<a href="https://colab.research.google.com/github/atedstone/unil_envi_ggl_hydrology_practicals/blob/main/gee_visualise_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualise and download Raster imagery with Google Earth Engine

UNIL MSc ENVI Glaciers and Glaciated Landscapes 2025

Authors: Christoph Posch, Andrew Tedstone

In [ ]:
import ee
import geemap.core as geemap
import geemap.colormaps as cm
import pandas as pd
import requests

In [ ]:
## Set up connections ##

# Authenticate GEE (log-in)
ee.Authenticate()

# Initialize Earth Engine, in the specific project for this course
ee.Initialize(project='unil-envi-ggl')

In [ ]:
# Find out user ID, we use this later to identify your tasks in the overall project task list
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
# Use the first part of the email address, minus the last four characters (to slightly obscure email address)
userid = gcloud_tokeninfo['email'].split('@')[0][:-4]
print(f'Your export tasks will be labelled with the user ID: {userid}')
tasks = []

## Search parameters

In [ ]:
## Specify parameters ##

# Area of interest (boundary coordinates, metres in EPSG:3413 'Polar Stereographic' projection)
x_min, y_min = -234080,-2538879
x_max, y_max = -155580,-2478879

# Collection (see https://developers.google.com/earth-engine/datasets)
# Note that we prefer not to use surface reflectance ('SR') as the processing to
# surface reflectance often introduces artefacts over ice sheet surfaces.
collection = 'COPERNICUS/S2_HARMONIZED'

# Date range (as 'YYYY-MM-DD')
start_date = '2023-07-05'
end_date = '2023-07-15'

# Bands (as 'B#')
bands = ['B12', 'B11', 'B8', 'B4', 'B3', 'B2']

# Max. cloudiness (# between 0..100), i.e., images with higher cloudiness are filtered out
max_cloud = 50

# Min. spatial coverage of aoi within satellite imagery (# between 0..100), i.e., images with lower coverage of the aoi within the scene are filtered out
spat_cov = 90

## Do the data search/filtering

In [ ]:
# Create aoi geometry
aoi = ee.Geometry.Polygon(
    coords=[[
        [x_min, y_min],
        [x_min, y_max],
        [x_max, y_max],
        [x_max, y_min],
        [x_min, y_min]
    ]],
    proj='EPSG:3413',
    geodesic=False
)

# Open dataset
dataset = (
    ee.ImageCollection(collection)
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud))
    .map(lambda image: image.clip(aoi))
)
data_list = dataset.toList(dataset.size())
print(f"Number of images in collection: {dataset.size().getInfo()}")
dataset

In [ ]:
## Specify vector layers

# aoi
aoi_vec = ee.FeatureCollection([ee.Feature(aoi)])
aoi_vec = aoi_vec.style(color='red', fillColor='00000000', width=2)

# Also load the Leverett catchment so that we can plot it in the maps below
catchm = ee.FeatureCollection("projects/unil-envi-ggl/assets/leverett_all_domain")
catchm = catchm.style(color='blue', fillColor= '00000000', width=1)

In [ ]:
## Spatial data filtering ##

# Calculate coverage
def compute_coverage(image):
    intersection = image.geometry().intersection(aoi, ee.ErrorMargin(10))
    coverage = ee.Number(intersection.area(1)) \
                  .divide(aoi.area(1)) \
                  .multiply(100)
    return image.set({'coverage': coverage})

# Calculate for each image in the collection
dataset_cov = dataset.map(compute_coverage)

# Filter by coverage
filt_dataset = dataset_cov.filter(
    ee.Filter.gte('coverage', spat_cov)
)
print(f"Number of images in filtered collection: {filt_dataset.size().getInfo()}")

filt_data_list = filt_dataset.toList(filt_dataset.size())

## RGB Visualization and Download

In [ ]:
## Data visualization ##

# Define bands (RGB) for visualization
vis_bands = ['B4', 'B3', 'B2']

# Visualization parameters
vis_params_rgb = {
    'bands': vis_bands,  # Red, Green, Blue
    'min': 0,      # Minimum reflectance value
    'max': 10000,  # Maximum reflectance value
    'gamma': 1.5   # Gamma correction
}

# Create map and center
m_rgb = geemap.Map() # possible arguments: layout={'height': '800px', 'width': '1200px'}
m_rgb.centerObject(aoi, 9)

# Add all satellite images
def add_rasters(list,vis_par,m,pre):
    for i in range(list.size().getInfo()):
        img = ee.Image(list.get(i))
        acquisition_date = img.get('system:time_start')
        formatted_date = ee.Date(acquisition_date).format('YYYY-MM-dd')
        m.addLayer(img, vis_par, f'{pre+formatted_date.getInfo()}')
add_rasters(filt_data_list, vis_params_rgb, m_rgb, "RGB_")

# Add vector layers (aoi, catchment)
m_rgb.addLayer(aoi_vec, {}, 'aoi')
m_rgb.addLayer(catchm, {}, 'catchments')

# Display map
m_rgb

In [ ]:
## Data download ##

# Define download parameters
path = "RGB img"
name_prefix = "RGB"
exp_bands = ['B4', 'B3', 'B2']


# Specify image export
def export_image(image, name):
    if name.startswith("RGB_"):
      im_exp = image.toInt16()
    else:
      im_exp = image.multiply(10000).toInt16()
    task = ee.batch.Export.image.toDrive(
        image=im_exp,
        description=f'{userid}_{name}',
        folder=path,
        fileNamePrefix=name,
        scale=10,
        region=aoi,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
    )
    task.start()
    print(f"{name}: STARTED")
    return task

def export_loop(list, name_pre, bnds):
    global tasks
    for i in range(list.size().getInfo()):
        image = ee.Image(list.get(i)).select(bnds)
        name = name_pre + "_" + ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        tasks.append(export_image(image, name))


# Export by looping through data list
export_loop(filt_data_list, name_prefix, exp_bands)

## Band Math

### NDWI ice, with download


In [ ]:
## Calculation ##

# Define name of new band
B_ndwi = "ndwi_ice"

# Calculate ndwi ice
def calc_ndwi(image):
    ndwi = image.normalizedDifference(['B2', 'B4']).rename(B_ndwi)
    return image.addBands(ndwi)

# Apply calculation to collection
ndwi_data = filt_dataset.map(calc_ndwi)
ndwi_data_list = ndwi_data.toList(ndwi_data.size())

In [ ]:
## Visualization ##

# Visualization parameters
vis_params_ndwi = {
    'bands': B_ndwi,  # Red, Green, Blue
    'min': 0,      # Minimum reflectance value
    'max': 0.2,  # Maximum reflectance value
    'palette': cm.palettes.Blues
}

# Create map and center
m_ndwi = geemap.Map() # possible arguments: layout={'height': '800px', 'width': '1200px'}
m_ndwi.centerObject(aoi, 9)

# OPTIONAL, add RGB images to map
#[m_ndwi.add_layer(layer) for layer in m_rgb.layers if layer.name.startswith("RGB_")]

# Add NDWI satellite images
add_rasters(ndwi_data_list, vis_params_ndwi, m_ndwi, "NDWI_")

# Add vector layers (aoi, catchment)
m_ndwi.addLayer(aoi_vec, {}, 'aoi')
m_ndwi.addLayer(catchm, {}, 'catchments')

# Display map
m_ndwi

In [ ]:
## Download ##

# Define download parameters
path = "NDWI img"
name_prefix = "NDWI"

# Export by looping through data list
export_loop(ndwi_data_list, name_prefix, B_ndwi)

### Albedo, with download

In [ ]:
## Calculation ##

# Define name of short-wave albedo band
B_alb = "albedo"

# Calculate albedos for each band
def calc_alb_all(image):
    band_names = image.bandNames()
    alb_bands = band_names.map(lambda b: image.select([b]).divide(10000).rename(ee.String(b).cat('_alb')))
    alb_image = ee.ImageCollection(alb_bands).toBands()
    return image.addBands(alb_image)

# Calculate short wave albedo (methodological approximation to https://doi.org/10.5194/tc-13-397-2019)
# N.b this band combination requires Sentinel-1 L1C collection, not L2A/SR!
def add_alb(image):
    albedo = image.expression(
        '0.346*alb2 + 0.130*alb4 + 0.373*alb8 + 0.085*alb11 + 0.072*alb12 - 0.0018',
        {
            'alb2': image.select('1_B2_alb'),
            'alb4': image.select('3_B4_alb'),
            'alb8': image.select('7_B8_alb'),
            'alb11': image.select('11_B11_alb'),
            'alb12': image.select('12_B12_alb'),
        },
    ).rename(B_alb)
    return image.addBands(albedo)

# Apply calculation to collection
alb_data = filt_dataset.map(calc_alb_all).map(add_alb)
alb_data_list = alb_data.toList(alb_data.size())

In [ ]:
## Visualization ##

# Visualization parameters
vis_params_alb = {
    'bands': B_alb,
    'min': 0,    # adjust to your expected ratio range
    'max': 1,
    'palette':cm.palettes.Greys_r
}

# Create map and center
m_alb = geemap.Map() # possible arguments: layout={'height': '800px', 'width': '1200px'}
m_alb.centerObject(aoi, 9)

# OPTIONAL, add RGB images to map
#[m_alb.add_layer(layer) for layer in m_rgb.layers if layer.name.startswith("RGB_")]

# Add Albedo satellite images
add_rasters(alb_data_list, vis_params_alb, m_alb, "Alb_")

# Add vector layers (aoi, catchment)
m_alb.addLayer(aoi_vec, {}, 'aoi')
m_alb.addLayer(catchm, {}, 'catchments')

# Display map
m_alb

In [ ]:
## Download ##

# Define download parameters
path = "Albedo img"
name_prefix = "Albedo"

# Export by looping through data list
export_loop(alb_data_list, name_prefix, B_alb)

## Task status and control

In [ ]:
# Check your tasks status
for task in tasks:
  s = task.status()
  # Comment out "if statement" if you (don't) want to have all cancelled tasks listed
  print(s['description'], s['state'], f"(ID: {task.id})") #if s['state'] != "CANCELLED" else None

In [ ]:
# If needed, cancel ALL tasks you have submitted by setting do_cancel to True and
# then running this cell.
do_cancel = False

# ------------------------------------------------------------------------------
if do_cancel:
  for task in tasks:
      task.cancel()
  print("All tasks cancelled")

In [ ]:
# If needed, cancel ONE of your submitted tasks
task_ID = ""   # insert ID from task list printed above (e.g TKEI6H45DVOITBG7XJDC2LT5 )

# ------------------------------------------------------------------------------
def cancel_task(task_id):
    tasks = ee.data.getTaskList()
    for t in tasks:
        if t['id'] == task_id:
            print("Cancelling:", t['description'], "ID:", t['id'])
            ee.data.cancelTask(t['id'])
            return
    print("Task not found:", task_id)

if task_ID != '':
  cancel_task(task_ID)